In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
data = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
data

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [3]:
df_scenario_1 = data.iloc[:17520]
df_scenario_2 = data.iloc[17520:35040]
df_scenario_3 = data.iloc[35040:52560]
df_scenario_4 = data.iloc[52560:70080]
df_scenario_5 = data.iloc[70080:]


In [6]:
train_set, test_set= np.split(df_scenario_1.node_1_demand, [int(.8 *len(df_scenario_1))])
print(train_set.shape)
print(test_set.shape)

# Fit a multivariate SARIMA model
order = (0, 1, 0)
seasonal_order = (1, 1, 1, 12)
model = sm.tsa.statespace.SARIMAX(train_set, order=order, seasonal_order=seasonal_order)
results = model.fit()

# Make predictions on the testing data
predictions = results.predict(start=test_set.index[0], end=test_set.index[-1])

# calculate the mean squared error
mae = mean_absolute_error(test_set, predictions)
rmse = sqrt(mean_squared_error(test_set, predictions))
r2 = r2_score(test_set, predictions)
print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

(14016,)
(3504,)


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)


MAE: 62.03061910309581
RMSE: 71.56336101023099
R2: -0.03629971648104058


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


In [9]:
train_set, test_set= np.split(df_scenario_1.node_1_pressure, [int(.8 *len(df_scenario_1))])
print(train_set.shape)
print(test_set.shape)

# Fit a multivariate SARIMA model
order = (0, 1, 0) # ARIMA order
seasonal_order = (1, 1, 1, 12) # seasonal order
model = sm.tsa.statespace.SARIMAX(train_set, order=order, seasonal_order=seasonal_order)
results = model.fit()

# Make predictions on the testing data
predictions = results.predict(start=test_set.index[0], end=test_set.index[-1])

# calculate the mean squared error
mae = mean_absolute_error(test_set, predictions)
rmse = sqrt(mean_squared_error(test_set, predictions))
r2 = r2_score(test_set, predictions)
print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

(14016,)
(3504,)


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


MAE: 0.12895458757877973
RMSE: 0.1578647367352608
R2: -0.011487937773262624


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


In [10]:
train_set, test_set= np.split(df_scenario_1.link_1_flow, [int(.8 *len(df_scenario_1))])
print(train_set.shape)
print(test_set.shape)

# Fit a multivariate SARIMA model
order = (0, 1, 0) # ARIMA order
seasonal_order = (1, 1, 1, 12) # seasonal order
model = sm.tsa.statespace.SARIMAX(train_set, order=order, seasonal_order=seasonal_order)
results = model.fit()

# Make predictions on the testing data
predictions = results.predict(start=test_set.index[0], end=test_set.index[-1])

# calculate the mean squared error
mae = mean_absolute_error(test_set, predictions)
rmse = sqrt(mean_squared_error(test_set, predictions))
r2 = r2_score(test_set, predictions)
print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

(14016,)
(3504,)


/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:527: ValueWarning: No frequency information was provided, so inferred frequency 30T will be used.
  % freq, ValueWarning)
/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:132: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  date_key = Timestamp(key, freq=base_index.freq)


MAE: 1184.7867897349781
RMSE: 1486.7884547922272
R2: -0.03320016919752078
